In [2]:
import pandas as pd
answers = open('./Answers.txt', 'w')

tweets =  pd.read_csv('./archive/2010.csv', index_col=False)
tweets
x = tweets.shape[0]
for i in range(2011, 2022):
    more_tweets = pd.read_csv(f'./archive/{str(i)}.csv', index_col=False)
    x += more_tweets.shape[0]
    tweets = pd.concat([tweets, more_tweets], ignore_index=True)

tweets.drop(['Unnamed: 0'], axis=1, inplace=True)
tweets.head()

,id,conversation_id,created_at,date,timezone,place,tweet,language,hashtags,cashtags,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,15434727182,15434727182,1.275676e+12,2010-06-04 18:31:57,0,NaN,"Please ignore prior tweets, as that was someon...",en,[],[],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
1,152153637639028736,152151847614943233,1.325111e+12,2011-12-28 22:27:08,0,NaN,@TheOnion So true :),en,[],[],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
2,151809315026636800,151809315026636800,1.325029e+12,2011-12-27 23:38:55,0,NaN,If you ever wanted to know the *real* truth ab...,en,[],[],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
3,151338939389706242,151338939389706242,1.324917e+12,2011-12-26 16:29:50,0,NaN,Walked around a neighborhood recently rebuilt ...,en,[],[],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
4,151337237429239808,151337237429239808,1.324917e+12,2011-12-26 16:23:04,0,NaN,"It was Xmas, so we brought presents for the ki...",en,[],[],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN


In [2]:
# Question 1
num_tweets = tweets.shape[0]
avg_nlikes = tweets['nlikes'].sum() / num_tweets
avg_nlikes = int(avg_nlikes)
answers.write(str(avg_nlikes) + '\n')

avg_nlikes

14705

In [3]:
# Question 2
total_views = (tweets['nlikes'].sum() + tweets['nreplies'].sum() + tweets['nretweets'].sum()) / 0.05
avg_view = total_views / num_tweets
avg_view = int(avg_view)
answers.write(str(avg_view) + '\n')

avg_view

338864

In [ ]:
# Question 3
from requests import get
from io import BytesIO
from PIL import Image

# get total number of char's in the tweet
tweet = tweets['tweet']
total_len_tweets = 0
for t in tweet:
    total_len_tweets += len(t)

# get total image size
def parse_url(photo_urls):
    urls = photo_urls[1:-1].split(', ')
    urls = list(map(lambda x:x[1:-1], urls))
    return urls

photos = tweets['photos']

photo_size_bytes = 0
sample_size = 0
total_num_photo = 0
for i, photo in enumerate(photos):
    if photo == '[]': 
        continue
        
    photo_urls = parse_url(photo)
    total_num_photo += len(photo_urls)

    #take some samples of photos to calculate the average photo size
    if i % 10 == 0: 
        for url in photo_urls:
            response = get(url)
            try:
                img = Image.open(BytesIO(response.content))
                width, height = img.size
                # jpg images has 3 bytes / pixel and photos are all jpg
                photo_size_bytes += width * height * 3 
                sample_size += 1
            except:
                pass
            
print('photo sample size:', sample_size)
avg_photo_size = photo_size_bytes / sample_size
total_photo_size_estimate = avg_photo_size * total_num_photo

# get total video size
VIDEO_SIZE = 256 * 2^20 # max Twitter video size is 512MB, so we assume average video size is 256MB
num_total_videos = 0


for i, video in enumerate(tweets['video']):
    if photos[i] != '[]' and parse_url(photos[i])[0] == tweets['thumbnail'][i]:
        num_total_videos += video
total_video_size_estimate = VIDEO_SIZE * num_total_videos

total_data_size = total_len_tweets + total_photo_size_estimate + total_video_size_estimate
avg_data_size_per_tweet = total_data_size / len(tweets)
avg_data_size_per_tweet = int(avg_data_size_per_tweet)

answers.write(str(avg_data_size_per_tweet) + '\n')
avg_data_size_per_tweet

In [ ]:
#Question 4
total_data_transfer = avg_data_size_per_tweet * total_views / 397000000
total_data_transfer = int(total_data_transfer)

answers.write(str(total_data_transfer) + '\n')
total_data_transfer

In [ ]:
#Question 5
ELEVEN_YEARS_IN_MIN = 11 * 365 * 24 * 60
avg_views_per_min = total_views / ELEVEN_YEARS_IN_MIN
avg_views_per_min = int(avg_views_per_min)

answers.write(str(avg_views_per_min) + '\n')
avg_views_per_min

In [ ]:
#Question 6
data_transfer_per_min = int(avg_views_per_min * avg_data_size_per_tweet)

answers.write(str(data_transfer_per_min) + '\n')
data_transfer_per_min

In [ ]:
answers.close()

In [6]:
# Question 7
print(tweets['nreplies'].sum() / (11 * 12))
"""
Whenever someone reply to a tweet, the funciton that increments "nreplies" will be executed.

And in total, Elon Musk has 146310 replies in his tweet per month.

Considering that he is 11th most popular person on Twitter in terms of followers,
an average person might get 1/10000 of Elon's replies.
|
That is 15 replies per month, and multiplying that by total user count of 397000000,
we get about 595 million modifications.

Assuming data type of reply count is 4 byte integer,
that is about 2.4 trillion bytes of data modification on a monthly basis
"""

146310.89393939395


'\nWhenever someone reply to a tweet, the funciton that increments "nreplies" will be executed.\n\nAnd in total, Elon Musk has 146310 replies in his tweet per month.\n\nConsidering that he is 11th most popular person on Twitter in terms of followers,\nan average person might get 1/10000 of Elon\'s replies.\n|\nThat is 15 replies per month, and multiplying that by total user count of 397000000,\nwe get about 595 million modifications.\n\nAssuming data type of reply count is 4 byte integer,\nthat is about 2.4 trillion bytes of data modification on a monthly basis\n'

5955000000